<a href="https://colab.research.google.com/github/aetev/Hearth-Stone-Python-Simulator/blob/main/card_enc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [142]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [143]:
import json
import tensorflow as tf
import re
from tensorflow.keras.layers import Embedding, TextVectorization
# Assuming AtomicCards.json is in your Google Drive, adjust the path if necessary
file_path = '/content/drive/MyDrive/MTGdata/AtomicCards.json'


with open(file_path, 'r') as f:
  data = json.load(f)['data']

In [144]:
def filter_legal_cards(data):
    legal_commander_data = {}
    for card_name, card_data in data.items():
        try:
            if card_data[0]['legalities']['commander'] == 'Legal':
                legal_commander_data[card_name] = card_data[0] # Store only the first element of card_data
        except (KeyError, IndexError):
            pass  # Or print a message, log it, etc.
    return legal_commander_data

filtered_data = filter_legal_cards(data)


In [145]:
items = list(filtered_data.items())

second_key, second_value = items[11]

text = second_value['text']
print(text)

Up to two target creatures each get +1/+0 and gain first strike until end of turn. (They deal combat damage before creatures without first strike.)
Madness {1}{R} (If you discard this card, discard it into exile. When you do, cast it for its madness cost or put it into your graveyard.)


In [148]:
class TextCleaningLayer(tf.keras.layers.Layer):
    def __init__(self,sentence_list, **kwargs):
        super(TextCleaningLayer, self).__init__(**kwargs)
        self.vectorizer = TextVectorization(standardize=None, ngrams=2, output_mode='int',output_sequence_length=150)
        self.vectorizer.adapt(self.clean_text(sentence_list))
        self.embedding_layer = Embedding(input_dim=50, output_dim=10)

    def clean_text(self, text):
        # Convert input to string tensor if it's not already
        if not isinstance(text, tf.Tensor):
            text = tf.convert_to_tensor(text, dtype=tf.string)

        cleaned_text = tf.strings.lower(text)
        cleaned_text = tf.strings.regex_replace(cleaned_text, '\n', '')
        cleaned_text = tf.strings.regex_replace(cleaned_text, '\.', ' ')
        cleaned_text = tf.strings.regex_replace(cleaned_text, '\{|\}', ' ')
        cleaned_text = tf.strings.regex_replace(cleaned_text, '\([^)]*\)', '')
        cleaned_text = tf.strings.regex_replace(cleaned_text, '[^\w\s+/\-]', '')
        cleaned_text = tf.strings.regex_replace(cleaned_text, '\s+', ' ')
        return cleaned_text

    def call(self, inputs):
        cleaned_text = self.clean_text(inputs)
        vectorized_text = self.vectorizer(cleaned_text)
        embedded_text = self.embedding_layer(vectorized_text)
        return embedded_text

# Example usage with a batch of texts
batch_texts = tf.constant([text])
cleaned_text_layer = TextCleaningLayer(batch_texts)
cleaned_batch = cleaned_text_layer.call(batch_texts)

for word in cleaned_batch[0]:
  print(word)



tf.Tensor(
[ 0.04454093 -0.04024317 -0.04562136 -0.04729252 -0.00391401  0.00048612
 -0.01235021 -0.00282847 -0.00149119 -0.03312128], shape=(10,), dtype=float32)
tf.Tensor(
[ 0.03377936 -0.0441206  -0.04406577  0.01921178 -0.02305237 -0.04099518
 -0.02147801  0.04582738 -0.02546657 -0.01239867], shape=(10,), dtype=float32)
tf.Tensor(
[ 0.04019291 -0.0132798   0.01261846  0.03375417 -0.02974938  0.02857881
  0.04996016  0.02158985  0.00710988  0.02978933], shape=(10,), dtype=float32)
tf.Tensor(
[-0.03383057 -0.04140841 -0.04346762  0.04081006 -0.03030522  0.03453082
 -0.03682553  0.0312036  -0.02672078 -0.04242401], shape=(10,), dtype=float32)
tf.Tensor(
[-0.00874938  0.04936918 -0.03555109 -0.00280114  0.02245587  0.0112871
 -0.04020103 -0.01474748  0.01170226 -0.00694202], shape=(10,), dtype=float32)
tf.Tensor(
[ 0.01415168 -0.01609625  0.00981603 -0.03798242  0.04734874  0.020143
 -0.00377606 -0.0255387   0.0286773  -0.01266656], shape=(10,), dtype=float32)
tf.Tensor(
[ 0.01119317  